# Get Started

Install the NebulaGrpah Python client:

In [ ]:
%pip install nebula3-python

Now connect to the Nebula Graph service by **ConnectionPool**:

In [2]:
# Default parameters
SERVER = "127.0.0.1"
PORT = 9669
USER = "root"
PASSWORD = "nebula"

In [5]:
import time
from nebula3.gclient.net import ConnectionPool
from nebula3.Config import Config

connection_pool = ConnectionPool()
# if the given servers are ok, return true, else return false
connection_pool.init([(SERVER, PORT)], Config())
# get a session with session_context, session will be released automatically
with connection_pool.session_context(USER, PASSWORD) as session:
    # Create the space
    session.execute_py("CREATE SPACE IF NOT EXISTS test(vid_type=FIXED_STRING(30));")
    time.sleep(20)  # two cycles of heartbeat, by default of a NebulaGraph cluster, we will need to sleep 20s
    # Create the schema
    session.execute_py(
        "USE test;"
        "CREATE TAG IF NOT EXISTS person(name string, age int);"
        "CREATE EDGE IF NOT EXISTS like (likeness double);"
    )
    time.sleep(20)  # two cycles of heartbeat, by default of a NebulaGraph cluster, we will need to sleep 20s
    # Clear the space if it exists
    session.execute_py("CLEAR SPACE test;")
    # Run queries with parameters
    args1 = {
        "p1": 3,
        "p2": True,
        "p3": "Bob",
    }
    stmt1 = "RETURN abs($p1)+3 AS col1, (toBoolean($p2) and false) AS col2, toLower($p3)+1 AS col3"
    res1 = session.execute_py(stmt1, args1)
    args2 = {
        "name1": "Bob",
        "age1": 10,
        "name2": "Lily",
        "age2": 9,
        "people": ["Bob", "Lily"],
    }
    session.execute_py(
        "INSERT VERTEX person(name, age) VALUES 'Bob':($name1, $age1), 'Lily':($name2, $age2)", 
        args2
    )
    stmt2 = "MATCH (v) WHERE id(v) in $people RETURN id(v) AS vertex_id"
    res2 = session.execute_py(stmt2, args2)

connection_pool.close()

Explore the results of the query:

In [5]:
res1.as_data_frame()

,col1,col2,col3
0,6,False,bob1


In [7]:
res2.as_data_frame()

,vertex_id
0,Lily
1,Bob


In [8]:
res1.as_primitive()

[{'col1': 6, 'col2': False, 'col3': 'bob1'}]

In [9]:
res2.as_primitive()

[{'vertex_id': 'Lily'}, {'vertex_id': 'Bob'}]

If the space is already created, you can connect to a specific space by **SessionPool**:

In [6]:
from nebula3.gclient.net.SessionPool import SessionPool
from nebula3.Config import SessionPoolConfig

session_pool = SessionPool(
    username=USER,
    password=PASSWORD,
    space_name="test",
    addresses=[(SERVER, PORT)]
    )
session_pool.init(SessionPoolConfig())
res3 = session_pool.execute_py("MATCH (v) RETURN v")

In [7]:
res3.as_data_frame()

,v
0,"{'vid': 'Bob', 'tags': {'person': {'age': 10, ..."
1,"{'vid': 'Lily', 'tags': {'person': {'age': 9, ..."


In [8]:
res3.as_primitive()

[{'v': {'vid': 'Bob', 'tags': {'person': {'age': 10, 'name': "Bob"}}}},
 {'v': {'vid': 'Lily', 'tags': {'person': {'age': 9, 'name': "Lily"}}}}]